<a href="https://colab.research.google.com/github/vlozg/speech_hmm/blob/main/Test_DiagHMM_011.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LƯU Ý: NOTEBOOK NÀY CHỈ DÙNG ĐỂ SHOW KẾT QUẢ CHẠY, KHÔNG NÊN CHẠY LẠI NOTEBOOK NÀY 
(VÌ TRONG NÀY CÓ CODE LƯU LẠI PRETRAINED MODEL LÊN DRIVE SẼ BỊ XÓA)

# Speech to text with HMM

- **Bài toán**: Chuyển giọng nói thành văn bản
    - **Input**: Đoạn ghi âm chứa nội dung là các số từ 0 đến 9
    - **Output**: Phân lớp của đoạn ghi âm

# Các biến thiết lập cho thử nghiệm

In [ ]:
n_mfcc_ceptrum = 12
n_delta_features = 1
n_mixtures = 1
fsdd_split = 0.3
wolfram_split = 0.3
experiment_id = '011'

In [ ]:
n_mfcc_features = n_mfcc_ceptrum * (1+n_delta_features)
n_mfcc_features

24

# Import và cài đặt thư viện

In [ ]:
# cài lib. note: cài xong phải restart runtime
!pip install pydub
!pip install pomegranate

In [ ]:
# Xác thực google để upload/download qua google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Quản lý file, folder
import os
from shutil import copyfile, rmtree
import random

# Xử lý audio
import librosa
import librosa.display
from scipy.io import wavfile

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pomegranate # Thư viện cho mô hình xác suất
from pomegranate import *

# Tải dữ liệu và lấy xác thực Google

Dữ liệu dùng để huấn luyện và đánh giá, còn xác thực google thì dùng để upload/download mô hình trên drive.

In [ ]:
%%capture
# download wolfram
if not os.path.isfile('./dataset_1_wolfram.zip'):
  !gdown --id 115tIAitBNeJC0DwrP-ZyJ6RS3TyWN0qD
  !unzip -o dataset_1_wolfram.zip

# dowload FSDD
if not os.path.isfile('./dataset_2_FSDD.zip'):
  !gdown --id 1Ua9zlPBc0Fv4xGHSQTb7eIvUh_dqFI6P
  !unzip -o dataset_2_FSDD.zip

# download self recorded audio
!gdown --id 1lH_k1AYMVlJvodtZdD7OK2zkdPXxlW9i

In [ ]:
# Lấy xác thực google để upload/download file
auth.authenticate_user()
gauth =  GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Hàm xử lý âm thanh

In [ ]:
def scaleAddNoise(wave):
  return ((wave - wave.mean()) / wave.std()) + 0.2*np.random.normal(size=wave.shape)

def read_process_audio(full_audio_path):
  # Bắt buộc âm thanh đọc vào phải cùng sample rate
  sample_rate, wave =  wavfile.read(full_audio_path)
  if sample_rate != 16000:
    wave, sample_rate =  librosa.load(full_audio_path, sr=16000) # Hàm đọc của librosa quá chậm, scipy nhanh gấp 5-10 lần
  else:
    wave=wave/32768 # Chuẩn hóa về số thực
  wave = np.pad(wave, (2000,2000), 'constant', constant_values=(0.0,0.0))
  wave = scaleAddNoise(wave)
  return wave, sample_rate

def unvoiced_frame(wave, sample_rate, min_len = 10):
  rms = librosa.feature.rms(y=wave)[0]
  r_normalized = (rms - np.mean(rms)) / np.std(rms)
  p = np.exp(r_normalized) / (1 + np.exp(r_normalized))
  thresh = 0.4
  slice_ = p > thresh
  while (slice_.sum() < 10):
    thresh-=0.05
    slice_ = p > thresh
  return slice_

def extract_mfcc(wave, sample_rate, trim=True):
  S = librosa.feature.melspectrogram(y=wave, sr=sample_rate, n_mels=40)
  if trim:
    S = S[:,unvoiced_frame(wave, sample_rate)]
  mfccs = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=n_mfcc_ceptrum+2, lifter=40)[2:,:].T/800
  return mfccs

def mfcc_delta_features(mfcc, order):
  if order==0:
    return mfcc
  dmfcc = librosa.feature.delta(mfcc, order=order)
  return dmfcc

def full_mfcc_from_file(full_audio_path, trim=True):
  wave, sr = read_process_audio(full_audio_path)
  mfccs = extract_mfcc(wave, sr, trim)
  full_mfccs = mfccs
  if n_delta_features >= 1:
    mfccs_d1 = mfcc_delta_features(mfccs, 1)
    full_mfccs = np.hstack([full_mfccs,mfccs_d1])
  if n_delta_features >= 2:
    mfccs_d2 = mfcc_delta_features(mfccs, 2)
    full_mfccs = np.hstack([full_mfccs,mfccs_d2])
  return full_mfccs

# Hàm bổ trợ

In [ ]:
def buildDataSet(dir, trim=True):
    # Filter out the wav audio files under the dir
    fileList = [f for f in os.listdir(dir) if os.path.splitext(f)[1] == '.wav']
    dataset = {}
    for fileName in fileList:
        tmp = fileName.split('.')[0]
        label = tmp.split('_')[1]

        # label = filename.split('_')[0]
        feature = full_mfcc_from_file(dir+fileName, trim)
        if label not in dataset.keys():
            dataset[label] = []
            dataset[label].append(feature)
        else:
            exist_feature = dataset[label]
            exist_feature.append(feature)
            dataset[label] = exist_feature
    return dataset

def makeTrainTestDir(mainFolder, filenames, rate):
    paths = [f'{mainFolder}',
             f'{mainFolder}/train_audio',
             f'{mainFolder}/test_audio']

    for path in paths:
        try:
            os.mkdir(path)
        except:
            rmtree(path)
            os.mkdir(path)

    random.seed(1)
    random.shuffle(filenames)
    splitPoint = int(len(filenames)*rate)
    trainFilenames = filenames[:splitPoint]
    testFilenames = filenames[splitPoint:]

    for filename in trainFilenames:
        copyfile(filename[0], f"{paths[1]}/{filename[1]}")

    for filename in testFilenames:
        copyfile(filename[0], f"{paths[2]}/{filename[1]}")

# Setup thư mục chứa data

In [ ]:
def formatFilenameFSDD(dir):
    filenames = []
    count = 0
    for filename in os.listdir(dir):
        tmp = str(count) + '_' + filename.split('_')[0] + '.wav'
        filenames.append((f"{dir}/{filename}", tmp))
        count += 1
    return filenames

filenames = formatFilenameFSDD('./dataset_2_FSDD')
makeTrainTestDir('fsdd', filenames, fsdd_split)

In [ ]:
def formatFilenameWolfram(dir):
    filenames = []
    count = 0
    folders = os.listdir(dir)
    for folder in folders:
        for filename in os.listdir(f'{dir}/{folder}'):
            tmp = str(count) + '_' + folder.split('_')[0] + '.wav'
            filenames.append((f"{dir}/{folder}/{filename}", tmp))
            count += 1

    return filenames

filenames = formatFilenameWolfram('./dataset_1_wolfram')
makeTrainTestDir('wolfram', filenames, wolfram_split)

# Huấn luyện mô hình (có thể bỏ qua vì mô hình đã save trên drive)

## Hàm train mô hình

In [ ]:
def Generate_DiagGMM(full_fset, n_features, n_states=5, n_cmps=3):
  means = np.mean(full_fset, axis=0)
  stds = np.std(full_fset, axis=0)
  # initial values for all gaussian components
  np.random.seed(None)
  dist_init = np.random.random((n_states, n_cmps, n_features, 2))
  dist_init[..., 0] -= 0.5  # center means to 0.0
  #for feat_i in range(n_features):
      # random init mean in range [-std, std)
      #dist_init[..., feat_i, 0] *= 2 * stds[feat_i]
      #dist_init[..., feat_i, 0] += means[feat_i]
      # random init std in range 1std/n_components
      #dist_init[..., feat_i, 1] *= stds[feat_i] / n_cmps

  dists = tuple(
      GeneralMixtureModel(list(
          IndependentComponentsDistribution(tuple(
              NormalDistribution(*dist_init[state_i, cmp_i, feat_i, :])
              for feat_i in range(n_features)
          ))
          for cmp_i in range(n_cmps)
      ))
      if n_cmps > 1 else
      IndependentComponentsDistribution(tuple(
          NormalDistribution(*dist_init[state_i, 0, feat_i, :])
          for feat_i in range(n_features)
      ))
      for state_i in range(n_states)
  )
  return dists

In [ ]:
# Generate progressive HMM model
def left_right_GMMHMM(seed_sample, x_dim, n_states=10, n_modals=9, random=0):
  rng = np.random.RandomState(random)
  if random!=0:
    init_prob = lambda: rng.rand(1)[0]
    
  model = HiddenMarkovModel()
  states = [ ]
  for i, state_model in enumerate(Generate_DiagGMM(seed_sample, x_dim, n_states, n_modals)):
    state = State( state_model, name=f"H{i}" )
    model.add_state(state)
    states.append(state)
  model.add_transition(model.start, states[0], 1)
  for i in range(n_states-1):
    model.add_transition(states[i], states[i], 0.5)
    model.add_transition(states[i], states[i+1], 0.5)
  model.add_transition(states[n_states-1], states[n_states-1], 0.5)
  model.add_transition(states[n_states-1], model.end, 0.5)
  model.bake()
  return model

In [ ]:
def train_GMMHMM(dataset, input_dim, n_hidden_state, n_gauss_modal):
    GMMHMM_Models = dict()

    for label in dataset.keys():
        print(f"Training model detect {label}")
        model = left_right_GMMHMM(dataset[label][0], input_dim, n_hidden_state, n_gauss_modal)
        model.fit(dataset[label], verbose=True, multiple_check_input=False)  # get optimal parameters
        GMMHMM_Models[label] = model

    return GMMHMM_Models

## Hàm test mô hình

In [ ]:
# test model
def evaluateModel(testDataset, model):
  if (len(testDataset) == 0):
    return
  digit_clf = BayesClassifier(list(dict(sorted(model.items())).values()))
  true_cnt = 0
  total = 0
  for label in sorted(testDataset.keys()):
      features = np.array(testDataset[label], dtype='object')
      pred = digit_clf.predict(features)
      iter_cnt = (pred == int(label)).sum()
      iter_total = len(features)
      total += iter_total
      true_cnt += iter_cnt
      print(f"{label}: {iter_cnt}/{iter_total} ({iter_cnt/iter_total})")
  print("Final recognition rate is %.2f"%(100.0*true_cnt/total), "%")

## Đọc và tiền xử lý data

In [ ]:
# prepare data for training
master_path = 'fsdd'

trainDir = master_path + '/train_audio/'
trainDataSet_fsdd = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_fsdd = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
# prepare data for training
master_path = 'wolfram'

trainDir = master_path + '/train_audio/'
trainDataSet_wolfram = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_wolfram = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
trainDataSet = trainDataSet_fsdd
if wolfram_split > 0:
  for label in trainDataSet_fsdd.keys():
    trainDataSet[label] = trainDataSet_fsdd[label] + trainDataSet_wolfram[label]

## **Mô hình 1**
- 10 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
for label in trainDataSet.keys():
  print(min(map(len, trainDataSet[label])))

10
10
10
10
10
10
10
10
10
10


In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,10,n_mixtures)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 21141901.40367448	Time (s): 0.2418
[2] Improvement: 12208.165866422234	Time (s): 0.2589
[3] Improvement: 6423.735125688952	Time (s): 0.2745
[4] Improvement: 2046.634781731991	Time (s): 0.2578
[5] Improvement: 1512.6750768704806	Time (s): 0.2586
[6] Improvement: 1932.6560436122818	Time (s): 0.2508
[7] Improvement: 1205.3362563955598	Time (s): 0.253
[8] Improvement: 674.9418783055153	Time (s): 0.2586
[9] Improvement: 390.3990010526031	Time (s): 0.2708
[10] Improvement: 219.75766453810502	Time (s): 0.2524
[11] Improvement: 107.78902944398578	Time (s): 0.2577
[12] Improvement: 78.6121629155241	Time (s): 0.2588
[13] Improvement: 72.09525338583626	Time (s): 0.2527
[14] Improvement: 33.23916156135965	Time (s): 0.2528
[15] Improvement: 21.674489035620354	Time (s): 0.2782
[16] Improvement: 16.20988361723721	Time (s): 0.2527
[17] Improvement: 13.433931039297022	Time (s): 0.2436
[18] Improvement: 7.836226260755211	Time (s): 0.2642
[19] Improvement: 4.64256

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (637/837)
2: (1094/1582)
6: (1706/2428)
8: (2244/3223)
0: (2889/4018)
7: (3475/4807)
9: (4065/5591)
5: (4741/6403)
3: (5351/7196)
4: (5987/7999)
Final recognition rate is 74.85 %
Evaluate on rest of FSDD set
0: (178/211)
1: (304/419)
3: (451/637)
9: (574/839)
7: (756/1073)
4: (961/1287)
5: (1158/1490)
8: (1271/1687)
2: (1399/1901)
6: (1492/2100)
Final recognition rate is 71.05 %
Evaluate on wolfram test set
4: (1245/1655)
8: (2355/3315)
1: (3566/4940)
2: (4578/6654)
6: (5821/8278)
0: (7148/9948)
3: (8399/11593)
7: (9613/13247)
5: (10959/14889)
9: (12234/16567)
Final recognition rate is 73.85 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_10_{n_mixtures}[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 2**
- 5 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,5,n_mixtures)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 30988617.84441623	Time (s): 0.1401
[2] Improvement: 14049.940447935602	Time (s): 0.1402
[3] Improvement: 3704.4527210913366	Time (s): 0.1567
[4] Improvement: 1471.8240588365588	Time (s): 0.1454
[5] Improvement: 749.8422647014959	Time (s): 0.1521
[6] Improvement: 455.7791122138733	Time (s): 0.155
[7] Improvement: 255.89191987889353	Time (s): 0.1519
[8] Improvement: 166.1896003440488	Time (s): 0.1429
[9] Improvement: 116.36997221258935	Time (s): 0.1646
[10] Improvement: 77.04929666104726	Time (s): 0.1421
[11] Improvement: 54.09189234219957	Time (s): 0.1415
[12] Improvement: 35.54845623765141	Time (s): 0.1478
[13] Improvement: 18.83277288195677	Time (s): 0.1477
[14] Improvement: 10.581746787880547	Time (s): 0.1422
[15] Improvement: 6.427073642495088	Time (s): 0.1507
[16] Improvement: 4.205418771132827	Time (s): 0.1489
[17] Improvement: 2.9100185317220166	Time (s): 0.1392
[18] Improvement: 2.0599052810575813	Time (s): 0.141
[19] Improvement: 1.45025

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (509/837)
2: (997/1582)
6: (1581/2428)
8: (2149/3223)
0: (2741/4018)
7: (3276/4807)
9: (3859/5591)
5: (4520/6403)
3: (4927/7196)
4: (5566/7999)
Final recognition rate is 69.58 %
Evaluate on rest of FSDD set
0: (184/211)
1: (276/419)
3: (370/637)
9: (479/839)
7: (635/1073)
4: (833/1287)
5: (1034/1490)
8: (1165/1687)
2: (1281/1901)
6: (1364/2100)
Final recognition rate is 64.95 %
Evaluate on wolfram test set
4: (1269/1655)
8: (2414/3315)
1: (3413/4940)
2: (4528/6654)
6: (5745/8278)
0: (6943/9948)
3: (7816/11593)
7: (8956/13247)
5: (10284/14889)
9: (11557/16567)
Final recognition rate is 69.76 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_5_{n_mixtures}[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 3**
- 3 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,3,n_mixtures)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 830888.6625227488	Time (s): 0.1076
[2] Improvement: 7352.899843739346	Time (s): 0.1051
[3] Improvement: 1698.0243519752985	Time (s): 0.1141
[4] Improvement: 407.4176171135623	Time (s): 0.1095
[5] Improvement: 91.9493538546376	Time (s): 0.1065
[6] Improvement: 916.9316527429037	Time (s): 0.09621
[7] Improvement: 7057.217176850303	Time (s): 0.1205
[8] Improvement: 5223.495171575923	Time (s): 0.09931
[9] Improvement: 2359.0285717539955	Time (s): 0.1092
[10] Improvement: 1157.1199687046465	Time (s): 0.1072
[11] Improvement: 535.4707542862743	Time (s): 0.1118
[12] Improvement: 261.8373248312855	Time (s): 0.09919
[13] Improvement: 139.8264628056204	Time (s): 0.1087
[14] Improvement: 85.84463160659652	Time (s): 0.1003
[15] Improvement: 49.55898144003004	Time (s): 0.09769
[16] Improvement: 28.075826703803614	Time (s): 0.108
[17] Improvement: 17.817502057529055	Time (s): 0.1022
[18] Improvement: 10.462027723086067	Time (s): 0.1265
[19] Improvement: 5.623

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (563/837)
2: (617/1582)
6: (1213/2428)
8: (1666/3223)
0: (2230/4018)
7: (2759/4807)
9: (3003/5591)
5: (3628/6403)
3: (4231/7196)
4: (4796/7999)
Final recognition rate is 59.96 %
Evaluate on rest of FSDD set
0: (149/211)
1: (252/419)
3: (406/637)
9: (441/839)
7: (595/1073)
4: (787/1287)
5: (978/1490)
8: (1068/1687)
2: (1070/1901)
6: (1174/2100)
Final recognition rate is 55.90 %
Evaluate on wolfram test set
4: (1159/1655)
8: (2061/3315)
1: (3150/4940)
2: (3262/6654)
6: (4493/8278)
0: (5669/9948)
3: (6846/11593)
7: (7956/13247)
5: (9227/14889)
9: (9859/16567)
Final recognition rate is 59.51 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_3_{n_mixtures}[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()